In [1]:
import dask
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns
import copy
import astropy
import hdbscan
import pandas as pd
from astropy.coordinates import Distance
from astropy import units as u
from astropy.cosmology import WMAP7
from astropy.io import fits
from collections import Counter
from pandas.api.types import is_numeric_dtype

In [2]:
y = pd.read_csv('iGrID_rcsed_8_clean.csv', index_col='id')
fltr = pd.notna(y.iGrID)

C:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
hdul = fits.open('rcsed_v2_8.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)

In [4]:
cols = hdul[1].columns

In [5]:
data = hdul[1].data

In [6]:
hdul.close()
del hdul

In [7]:
DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder())
del data

In [8]:
DATA.shape

(4109726, 479)

In [9]:
type_stats = {}
for c in DATA.columns:
    t = DATA[c].dtype
    if t in type_stats:
        type_stats[DATA[c].dtype] += 1
    else:
        type_stats[DATA[c].dtype] = 1

print(type_stats)

{dtype('<i8'): 2, dtype('float64'): 410, dtype('int32'): 10, dtype('int16'): 9, dtype('O'): 9, dtype('float32'): 37, dtype('uint8'): 2}


In [10]:
for c in DATA.columns:
    t = DATA[c].dtype
    if t == '<i8':
        print(c)

ind
bestObjID_sdss


In [11]:
DATA.drop(['ind', 'bestObjID_sdss'], inplace=True, axis=1)
DATA = DATA.select_dtypes(include='number')

In [12]:
DATA.shape

(4109726, 468)

In [13]:
type_stats = {}
for c in DATA.columns:
    t = DATA[c].dtype
    if t in type_stats:
        type_stats[DATA[c].dtype] += 1
    else:
        type_stats[DATA[c].dtype] = 1

print(type_stats)

{dtype('float64'): 410, dtype('int32'): 10, dtype('int16'): 9, dtype('float32'): 37, dtype('uint8'): 2}


In [14]:
for c in DATA.columns:
    print(c, ' | ', DATA[c].isna().sum()/DATA.shape[0] , '% |', DATA[fltr][c].isna().sum()/DATA[fltr].shape[0], '%')

ra  |  0.0 % | 0.0 %
dec  |  0.0 % | 0.0 %
z_sdss  |  0.232334710391885 % | 0.0 %
zErr_sdss  |  0.23176581601790483 % | 0.0 %
specObjID_sdss  |  0.23176581601790483 % | 0.0 %
bossSpecObjID_sdss  |  0.0 % | 0.0 %
mjd_sdss  |  0.0 % | 0.0 %
plate_sdss  |  0.0 % | 0.0 %
fiberID_sdss  |  0.0 % | 0.0 %
z_2df  |  0.9463618742466042 % | 0.9594772545889865 %
q_z_2df  |  0.0 % | 0.0 %
z_6df  |  0.9717256089578721 % | 0.9912983838786912 %
quality_6df  |  0.0 % | 0.0 %
specid_6df  |  0.0 % | 0.0 %
recno_uzc  |  0.0 % | 0.0 %
z_uzc  |  0.9952880070350189 % | 0.9764864325618515 %
z_err_uzc  |  0.9952865470836741 % | 0.9764764565043894 %
z_hectospec  |  0.9612275854886676 % | 0.9952015163607343 %
z_cfa  |  0.9936032231832487 % | 0.9754165003990423 %
z_err_cfa  |  0.9936095496390757 % | 0.9754239824421389 %
z_lamost  |  0.9559082527642963 % | 0.8736033519553073 %
z_err_lamost  |  0.9452097293104211 % | 0.8335370111731844 %
obsid_lamost  |  0.0 % | 0.0 %
lmjd_lamost  |  0.0 % | 0.0 %
mjd_lamost  |  0.

integmag_r_legacy  |  0.07372754290675339 % | 0.005020450917797286 %
integmag_z_legacy  |  0.07375528198230247 % | 0.0050104748603351955 %
kronmag_NUV_galex  |  0.6486089340262586 % | 0.29613427773343975 %
kronmagerr_NUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
kronrad_NUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
kronmag_FUV_galex  |  0.663627210183842 % | 0.3014390462889066 %
kronmagerr_FUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
kronrad_FUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
aper2mag_u_sdss  |  0.19692286055080072 % | 0.0 %
aper2magerr_u_sdss  |  0.19692286055080072 % | 0.0 %
aper2mag_g_sdss  |  0.19692286055080072 % | 0.0 %
aper2magerr_g_sdss  |  0.19692286055080072 % | 0.0 %
aper2mag_r_sdss  |  0.19692286055080072 % | 0.0 %
aper2magerr_r_sdss  |  0.19692286055080072 % | 0.0 %
aper2mag_i_sdss  |  0.19692286055080072 % | 0.0 %
aper2magerr_i_sdss  |  0.19692286055080072 % | 0.0 %
aper2mag_z_sdss  |  0.19692286055080072 % | 0.0

aper3mag_NUV_galex  |  0.6486089340262586 % | 0.29613427773343975 %
aper3magerr_NUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
aper3mag_FUV_galex  |  0.663627210183842 % | 0.3014390462889066 %
aper3magerr_FUV_galex  |  0.6032338895585739 % | 0.2630387071029529 %
aper3mag_w1_wise  |  0.5380872106802254 % | 0.5525588587390263 %
aper3magerr_w1_wise  |  0.536393423795163 % | 0.5520650438946528 %
aper3mag_w2_wise  |  0.5380872106802254 % | 0.5525588587390263 %
aper3magerr_w2_wise  |  0.536393423795163 % | 0.5520650438946528 %
aper3mag_w3_wise  |  0.5380872106802254 % | 0.5525588587390263 %
aper3magerr_w3_wise  |  0.536393423795163 % | 0.5520650438946528 %
aper3mag_w4_wise  |  0.5380872106802254 % | 0.5525588587390263 %
aper3magerr_w4_wise  |  0.536393423795163 % | 0.5520650438946528 %
aper6mag_U_vst  |  0.9765619411123758 % | 0.989036312849162 %
aper6magerr_U_vst  |  0.9765619411123758 % | 0.989036312849162 %
aper6mag_G_vst  |  0.9452995163181195 % | 0.9832551875498803 %
aper6mag

kcorr_aper2mag_H_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_ks_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_g_legacy  |  0.49283626207683917 % | 0.03587639664804469 %
kcorr_aper2mag_r_legacy  |  0.49283626207683917 % | 0.03587639664804469 %
kcorr_aper2mag_z_legacy  |  0.5038024432772403 % | 0.05609537110933759 %
kcorr_aper2mag_u_sdss  |  0.5746614251169057 % | 3.741021548284118e-05 %
kcorr_aper2mag_g_sdss  |  0.5746614251169057 % | 3.741021548284118e-05 %
kcorr_aper2mag_r_sdss  |  0.5746614251169057 % | 3.741021548284118e-05 %
kcorr_aper2mag_i_sdss  |  0.5746614251169057 % | 3.741021548284118e-05 %
kcorr_aper2mag_z_sdss  |  0.5746614251169057 % | 3.741021548284118e-05 %
kcorr_aper2mag_Y_ukidss  |  0.8327749830523982 % | 0.6255536711891461 %
kcorr_aper2mag_J_ukidss  |  0.8327749830523982 % | 0.6255536711891461 %
kcorr_aper2mag_H_ukidss  |  0.8327749830523982 % | 0.6255536711891461 %
kcorr_aper2mag_K_ukidss  |  0.8327749830523982 % | 0.62555367

KeyboardInterrupt: 

In [ ]:
for c in DATA.columns:
    print(c, DATA[c].value_counts().head(5))
    print(DATA[c].min(), DATA[c].median(), DATA[c].max())
    print()

In [15]:
DATA.replace([-2147483648, -9223372036854775808,-32768,255, -999999488.0,
 99.0,
 -99.0,
 9999.0,
 -999.0,
 float('inf'),
 -9999.0], np.nan, inplace=True)

In [16]:
for_del = []
for c in DATA.columns:
    print(c, ' | ', DATA[c].isna().sum()/DATA.shape[0] , '% |', DATA[fltr][c].isna().sum()/DATA[fltr].shape[0], '%')
    if (DATA[c].isna().sum()/DATA.shape[0] > 0.35):
        for_del.append(c)

ra  |  0.0 % | 0.0 %
dec  |  0.0 % | 0.0 %
z_sdss  |  0.232334710391885 % | 0.0 %
zErr_sdss  |  0.23176581601790483 % | 0.0 %
specObjID_sdss  |  0.23176581601790483 % | 0.0 %
bossSpecObjID_sdss  |  0.23176630266835307 % | 0.0 %
mjd_sdss  |  0.23176581601790483 % | 0.0 %
plate_sdss  |  0.23176581601790483 % | 0.0 %
fiberID_sdss  |  0.23374794329354318 % | 0.003132482043096568 %
z_2df  |  0.9463618742466042 % | 0.9594772545889865 %
q_z_2df  |  0.9456533111939823 % | 0.9563397845171588 %
z_6df  |  0.9717256089578721 % | 0.9912983838786912 %
quality_6df  |  0.9717088195174082 % | 0.9912509976057462 %
specid_6df  |  0.9717093061678564 % | 0.9912509976057462 %
recno_uzc  |  0.9952656211144003 % | 0.9763193335993615 %
z_uzc  |  0.9952880070350189 % | 0.9764864325618515 %
z_err_uzc  |  0.9952865470836741 % | 0.9764764565043894 %
z_hectospec  |  0.9612275854886676 % | 0.9952015163607343 %
z_cfa  |  0.9936032231832487 % | 0.9754165003990423 %
z_err_cfa  |  0.9936095496390757 % | 0.97542398244213

kronmag_u_kids  |  0.943471900559794 % | 0.9720695331205108 %
kronmagerr_u_kids  |  0.9434804169426381 % | 0.9720695331205108 %
kronmag_g_kids  |  0.9411459547424816 % | 0.9720445929768555 %
kronmagerr_g_kids  |  0.9411461980677057 % | 0.9720445929768555 %
kronmag_r_kids  |  0.9412561810690055 % | 0.9722565841979249 %
kronmagerr_r_kids  |  0.9412561810690055 % | 0.9722565841979249 %
kronmag_i_kids  |  0.9412002162674592 % | 0.9720595570630487 %
kronmagerr_i_kids  |  0.9412002162674592 % | 0.9720595570630487 %
kronrad_kids  |  0.9410250221060966 % | 0.9719722665602554 %
integmag_g_legacy  |  0.09072259318504446 % | 0.03265413008778931 %
integmag_r_legacy  |  0.08924706902601293 % | 0.029738627294493215 %
integmag_z_legacy  |  0.09212487645161746 % | 0.04135574620909816 %
kronmag_NUV_galex  |  0.6486089340262586 % | 0.29613427773343975 %
kronmagerr_NUV_galex  |  0.615678514820696 % | 0.28260674381484435 %
kronrad_NUV_galex  |  0.6156768115441272 % | 0.28260674381484435 %
kronmag_FUV_gale

aper3mag_h_vhs  |  0.8978031625466029 % | 0.9309482242617717 %
aper3magerr_h_vhs  |  0.8978031625466029 % | 0.9309482242617717 %
aper3mag_ks_vhs  |  0.8978031625466029 % | 0.9309482242617717 %
aper3magerr_ks_vhs  |  0.8978031625466029 % | 0.9309482242617717 %
aper3mag_g_legacy  |  0.09931489349898266 % | 0.029127593774940144 %
aper3magerr_g_legacy  |  0.09931489349898266 % | 0.029127593774940144 %
aper3mag_r_legacy  |  0.09981419685886601 % | 0.025354150039904228 %
aper3magerr_r_legacy  |  0.09981419685886601 % | 0.025354150039904228 %
aper3mag_z_legacy  |  0.10170386054934076 % | 0.0424456304868316 %
aper3magerr_z_legacy  |  0.10170386054934076 % | 0.0424456304868316 %
aper3mag_NUV_galex  |  0.6486089340262586 % | 0.29613427773343975 %
aper3magerr_NUV_galex  |  0.6165892811345574 % | 0.2839684756584198 %
aper3mag_FUV_galex  |  0.663627210183842 % | 0.3014390462889066 %
aper3magerr_FUV_galex  |  0.8239746396718419 % | 0.5397072027134876 %
aper3mag_w1_wise  |  0.5380872106802254 % | 0.5

kcorr_petromag_i_skymapper  |  0.9472174057345916 % | 0.8959572027134877 %
kcorr_petromag_z_skymapper  |  0.921934941648178 % | 0.8740248403830806 %
kcorr_petromag_Y_viking  |  0.9556817169806454 % | 0.9514465283320032 %
kcorr_petromag_J_viking  |  0.9556817169806454 % | 0.9514465283320032 %
kcorr_petromag_H_viking  |  0.9556817169806454 % | 0.9514465283320032 %
kcorr_petromag_ks_viking  |  0.9556817169806454 % | 0.9514465283320032 %
kcorr_petromag_Z_viking  |  0.9817262756689862 % | 0.9514465283320032 %
kcorr_aper2mag_Y_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_J_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_H_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_ks_vhs  |  0.928002256111478 % | 0.9309557063048683 %
kcorr_aper2mag_g_legacy  |  0.49283626207683917 % | 0.03587639664804469 %
kcorr_aper2mag_r_legacy  |  0.49283626207683917 % | 0.03587639664804469 %
kcorr_aper2mag_z_legacy  |  0.5038024432772403 % | 0.0560953711093375

In [ ]:
print(len(for_del))

In [17]:
DATA.drop(for_del, axis=1, inplace=True)

In [ ]:
DATA.shape

In [18]:
cmn = DATA.columns
for_del = []

In [19]:
for i in range(len(cmn)):
    if cmn[i] in for_del:
        continue
    for j in range(i + 1, len(cmn)):
        if cmn[j] in for_del:
            continue
        if abs(DATA[cmn[i]].corr(DATA[cmn[j]])) > 0.9:
            for_del.append(cmn[j])

In [ ]:
print(for_del)

In [20]:
DATA.drop(for_del, axis=1, inplace=True)

In [ ]:
DATA.shape

In [ ]:
corrmat = DATA.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(40,40))
g=sns.heatmap(DATA[top_corr_features].corr(),annot=True,cmap="RdYlGn")
plt.savefig('V8corrFIltered.png')
del corrmat, top_corr_features, g

In [21]:
print(DATA.columns)

Index(['ra', 'dec', 'z_sdss', 'zErr_sdss', 'specObjID_sdss', 'fiberID_sdss',
       'petromag_u_sdss', 'petromagerr_u_sdss', 'petror50_u_sdss',
       'petromagerr_g_sdss', 'petror50_g_sdss', 'petromagerr_r_sdss',
       'petror50_r_sdss', 'petromagerr_i_sdss', 'petromagerr_z_sdss',
       'petror50_z_sdss', 'kronmag_g_panstarrs', 'kronmagerr_g_panstarrs',
       'kronmag_r_panstarrs', 'kronmagerr_r_panstarrs', 'kronmag_i_panstarrs',
       'kronmagerr_i_panstarrs', 'kronmag_z_panstarrs',
       'kronmagerr_z_panstarrs', 'kronmag_y_panstarrs',
       'kronmagerr_y_panstarrs', 'integmag_g_legacy', 'integmag_z_legacy',
       'aper2magerr_u_sdss', 'aper2magerr_g_sdss', 'aper2magerr_r_sdss',
       'aper2magerr_i_sdss', 'aper2magerr_z_sdss', 'aper2magerr_g_legacy',
       'aper2magerr_r_legacy', 'aper2magerr_z_legacy', 'aper3magerr_u_sdss',
       'aper3magerr_g_sdss', 'aper3magerr_r_sdss', 'aper3magerr_i_sdss',
       'aper3magerr_z_sdss', 'aper3magerr_g_legacy', 'aper3mag_r_legacy',
   